In [ ]:
%matplotlib inline
from pprint import pprint

import plot
from simulate import simulate_withdrawals
import harvesting
import market
import withdrawal
import metrics

from decimal import Decimal

import itertools
import math
import pandas

In [ ]:
YEARS = 25

In [ ]:
def run_comparison(series):
    (r1, r2) = itertools.tee(series)
    p1 = (1214000 * .6, 1214000 * .4)
    p2 = (2459000, 0)
    
    sim1 = simulate_withdrawals(r1, years=YEARS, harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=withdrawal.VPW, portfolio=p1)
    sim2 = simulate_withdrawals(r2, years=YEARS, harvesting=harvesting.N_100_RebalanceHarvesting, withdraw=withdrawal.VPW, portfolio=p2)

    s1 = [n.withdraw_r for n in sim1]
    s2 = [n.withdraw_r for n in sim2]
    
    return (s1, s2)

In [ ]:
#MARKET = market.Returns_US_1871()
#MARKET = market.PortfolioCharts_1927(market.PortfolioCharts_1927.Weights(LCB=.8, SCV=.2))
MARKET = market.Japan_1957()

RETIRE_YEAR = 1990
s1, s2, = run_comparison(MARKET.iter_from(RETIRE_YEAR))
print("60/40", int(metrics.cew(s1)))
print("100/0", int(metrics.cew(s2)))
plot.plot_n({'60/40' : s1, '100/0': s2}, 'Year', '%d Retirement' % RETIRE_YEAR)

In [ ]:
incomes_60 = []
incomes_100 = []

for i in range(MARKET.start_year, 2016-YEARS):
    s1, s2 = run_comparison(MARKET.iter_from(i))
    incomes_60 += [int(n) for n in s1]
    incomes_100 += [int(n) for n in s2]
    
incomes_60 = pandas.Series(data=incomes_60)
incomes_100 = pandas.Series(data=incomes_100)
diff = incomes_100 - incomes_60
print('60', incomes_60.quantile(.01))
print('100', incomes_100.quantile(.01))

df = pandas.DataFrame(data={'60/40' : incomes_60, '100/0' : incomes_100, 'Delta' : diff})

In [ ]:
def index_to_year(n):
    year_of_retirement = n//YEARS + 1871
    year_of_income = n%YEARS + year_of_retirement
    return "%d/%d" % (year_of_retirement, year_of_income)

df['Income date'] = df.index.map(index_to_year)
df.head()

In [ ]:
low_income = df.loc[(df['60/40'] < 40000) | (df['100/0'] < 40000)]

win_100 = low_income.sort_values('Delta', ascending=False)
win_60 = low_income.sort_values('Delta', ascending=True)

print(win_60.head())